In [8]:
#also ref: https://www.tensorflow.org/tutorials/representation/word2vec

In [6]:
import collections
import numpy as np
import tensorflow as tf
import matplotlib.pyplot as plt
%matplotlib inline

BARCH_SIZE = 20
#Dimension of the embedding vector. Two too small to get
#any meaningful embeddings, but let's make it 2 for simple visualization
EMBEDDING_SIZE = 2
NUM_SAMPLED = 15 #Number of negative examples to sample

#Sample sentences
sentences = ["the quick brown fox jumped over the lazy dog",
            "I love cats and dogs",
            "we all love cats and dogs",
            "cats and dogs are great",
            "sung likes cats",
            "she loves dogs",
            "cats can be very independent",
            "cats are great companions when they want to be",
            "cats are playful",
            "cats are natural hunters",
            "It's raining cats and dogs",
            "dogs and cats love sung"]

#sentences to words and count
words = " ".join(sentences).split()
count = collections.Counter(words).most_common()
print("Word count", count[:5])

#Build dictionaries
rdic = [i[0] for i in count] #reverse dic, idx -> word
dic = {w: i for i, w in enumerate(rdic)} #dic, word -> id
voc_size = len(dic)

#Make indexed word data
data = [dic[word] for word in words]
print("Sample data", data[:10], [rdic[t] for t in data[:10]])


#Let's make a training data for window size 1 for simplicity
# ([the, brown], quick), ([quick, fox], brown), ([brown, jumped], fox), ...

Word count [('cats', 10), ('dogs', 6), ('and', 5), ('are', 4), ('love', 3)]
Sample data [5, 9, 10, 11, 12, 13, 5, 14, 15, 16] ['the', 'quick', 'brown', 'fox', 'jumped', 'over', 'the', 'lazy', 'dog', 'I']
